<a href="https://colab.research.google.com/github/palsreejib/Vehicle-Cut-in-Detection/blob/main/PS-08_Vehicle_Cut_in_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade torch ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 799.6/799.6 kB 55.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-many

In [ ]:
!pip install opencv-python

In [ ]:
import cv2
from ultralytics import YOLO, solutions
import numpy as np

# Load the YOLOv8 model
model = YOLO("yolov8n.pt")
names = model.model.names

cap = cv2.VideoCapture("test_vid.mp4")
assert cap.isOpened(), "Error reading video file"
w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))

# Video writer
video_writer = cv2.VideoWriter("distance_speed_calculation.avi", cv2.VideoWriter_fourcc(*"mp4v"), fps, (w, h))

# Init distance-calculation obj
dist_obj = solutions.DistanceCalculation(names=names, view_img=True)

# Dictionary to store previous centroids
prev_centroids = {}

while cap.isOpened():
    success, im0 = cap.read()
    if not success:
        print("Video frame is empty or video processing has been successfully completed.")
        break

    tracks = model.track(im0, persist=True, show=False)
    im0 = dist_obj.start_process(im0, tracks)

    # Calculate speed
    current_centroids = {}
    for track in tracks:
        track_id = track['id']
        centroid = track['bbox'].center  # Assuming bbox has a center method
        current_centroids[track_id] = centroid

        if track_id in prev_centroids:
            prev_centroid = prev_centroids[track_id]
            distance = np.linalg.norm(np.array(centroid) - np.array(prev_centroid))
            speed = distance * fps  # Speed in pixels per second
            cv2.putText(im0, f"Speed: {speed:.2f} px/s | Time to collision {distance/speed}", (int(centroid[0]), int(centroid[1]) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

    prev_centroids = current_centroids
    video_writer.write(im0)

cap.release()
video_writer.release()
cv2.destroyAllWindows()

WARNING ⚠️ Environment does not support cv2.imshow() or PIL Image.show()


0: 384x640 2 persons, 4 cars, 1 bus, 279.6ms
Speed: 15.7ms preprocess, 279.6ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)


TypeError: new(): invalid data type 'str'